##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Kubernetes를 사용한 고성능 시뮬레이션

---

이 튜토리얼에서는 Kubernetes에 배포된 TFF 런타임을 사용하여 고성능 시뮬레이션을 설정하는 방법을 설명합니다.

시연을 위해 [이미지 분류를 위한 페더레이션 학습](https://www.tensorflow.org/federated/tutorials/federated_learning_for_image_classification) 튜토리얼의 이미지 분류를 위한 TFF 시뮬레이션을 사용하지만 Kubernetes에서 실행되는 두 개의 TFF 작업자로 구성된 다중 시스템 환경에 대해 실행합니다. 훈련을 위해 동일한 [EMNIST 데이터세트](https://www.tensorflow.org/federated/tutorials/federated_learning_for_image_classification#preparing_the_input_data)를 사용하지만 각 TFF 작업자에 하나씩 두 개의 파티션으로 나뉩니다.

이 튜토리얼은 다음 Google Cloud 서비스를 참조합니다.

- [GKE](https://cloud.google.com/kubernetes-engine/) - Kubernetes 클러스터를 생성하지만 클러스터 생성 후의 모든 단계는 모든 Kubernetes 설치에서 사용할 수 있습니다.
- [Filestore](https://cloud.google.com/filestore) - 훈련 데이터를 제공하지만 Kubernetes [영구 볼륨](https://kubernetes.io/docs/concepts/storage/persistent-volumes/)으로 마운트할 수 있는 모든 저장 매체와 함께 작동합니다.

> **참고:** 이 튜토리얼에서는 기존 GCP 프로젝트가 있다고 가정합니다.

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/federated/tutorials/high_performance_simulation_with_kubernetes"><img src="https://www.tensorflow.org/images/tf_logo_32px.png"> TensorFlow.org에서 보기</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/federated/tutorials/high_performance_simulation_with_kubernetes.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a>
</td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/federated/tutorials/high_performance_simulation_with_kubernetes.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소그 보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/federated/tutorials/high_performance_simulation_with_kubernetes.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드하기</a></td>
</table>

## Kubernetes에서 TFF 작업자 실행하기

### Kubernetes 클러스터 생성하기

[worker_service.py](https://github.com/tensorflow/federated/blob/main/docs/tutorials/high_performance_simulation_with_kubernetes/worker_service.py)에는 맞춤형 TFF 작업자의 소스 코드가 포함되어 있습니다. 데이터세트 파티션을 로드하고 페더레이션 학습의 각 라운드에서 샘플링하기 위한 맞춤형 로직으로 시뮬레이션 서버를 실행합니다. 자세한 내용은 [TFF에서 원격 데이터 로드](https://www.tensorflow.org/federated/tutorials/loading_remote_data)를 참조하세요.

여기서는 TFF 작업자를 Kubernetes에 컨테이너화된 애플리케이션으로 배포할 것입니다. Docker 이미지를 빌드하는 것으로 시작하겠습니다. 이 [Dockerfile](https://github.com/tensorflow/federated/blob/main/docs/tutorials/high_performance_simulation_with_kubernetes/Dockerfile)을 사용하고 다음을 실행하여 코드를 패키징할 수 있습니다.

```
$ WORKER_IMAGE=tff-worker-service:latest

$ docker build --tag $WORKER_IMAGE --file "./Dockerfile" .
```

([worker_service.py](https://github.com/tensorflow/federated/blob/main/docs/tutorials/high_performance_simulation_with_kubernetes/worker_service.py) 및 [Dockerfile](https://github.com/tensorflow/federated/blob/main/docs/tutorials/high_performance_simulation_with_kubernetes/Dockerfile)이 작업 디렉터리에 있다고 가정합니다.)

그런 다음 생성하려는 Kubernetes 클러스터에서 액세스할 수 있는 컨테이너 리포지토리에 이미지를 게시합니다.

```
$ docker push $WORKER_IMAGE
```

### Kubernetes 클러스터 생성하기

The following step only needs to be done once. The cluster can be re-used for future workloads.

GKE 안내에 따라 Filestore CSI 드라이버가 설정된 상태로 [클러스터를 만듭니다](https://cloud.google.com/kubernetes-engine/docs/how-to/persistent-volumes/filestore-csi-driver#enabling_the_on_a_new_cluster). 예를 들면 다음과 같습니다.

```
gcloud container clusters create tff-cluster --addons=GcpFilestoreCsiDriver
```

The commands to interact with GCP can be run [locally](https://cloud.google.com/kubernetes-engine/docs/tutorials/hello-app#option_b_use_command-line_tools_locally) or in the [Google Cloud Shell](https://cloud.google.com/shell/). We recommend the Google Cloud Shell since it doesn't require additional setup.

이 튜토리얼의 나머지 부분에서는 클러스터 이름이 `tff-cluster`라고 가정하지만 실제 이름은 중요하지 않습니다.

### Deploy the TFF Worker Application

[worker_deployment.yaml](https://github.com/tensorflow/federated/blob/main/docs/tutorials/high_performance_simulation_with_kubernetes/worker_deployment.yaml)은 각각 두 개의 복제본이 있는 자체 Kubernetes 포드에 2개의 TFF 작업자를 세우기 위한 구성을 선언합니다. 이 구성을 실행 중인 클러스터에 적용할 수 있습니다.

```
kubectl apply -f worker_deployment.yaml
```

변경이 요청되면 포드가 준비되었는지 확인할 수 있습니다.

```
kubectl get pod
NAME                                        READY   STATUS    RESTARTS   AGE
tff-workers-deployment-1-6bb8d458d5-hjl9d   1/1     Running   0          5m
tff-workers-deployment-1-6bb8d458d5-jgt4b   1/1     Running   0          5m
tff-workers-deployment-2-6cb76c6f5d-hqt88   1/1     Running   0          5m
tff-workers-deployment-2-6cb76c6f5d-xk92h   1/1     Running   0          5m
```

각 작업자 인스턴스는 엔드포인트가 있는 로드 밸런서 뒤에서 실행됩니다. 로드 밸런서의 외부 IP 주소를 조회합니다.

```
kubectl get service
NAME                    TYPE           CLUSTER-IP    EXTERNAL-IP     PORT(S)        AGE
tff-workers-service-1   LoadBalancer   XX.XX.X.XXX   XX.XXX.XX.XXX   80:31830/TCP   6m
tff-workers-service-2   LoadBalancer   XX.XX.X.XXX   XX.XXX.XX.XXX   80:31319/TCP   6m
```

이는 나중에 훈련 루프를 실행 중인 작업자에 연결하는 데 필요합니다.

> **참고:** 이렇게 하면 배포가 인터넷에 노출되며 데모용으로만 사용됩니다. 운영 용도의 경우, 방화벽과 인증을 강력히 권장합니다.

## 훈련 데이터 준비

훈련에 사용할 EMNIST 파티션은 TFF의 공개 [데이터세트 리포지토리](https://console.cloud.google.com/storage/browser/tff-datasets-public/emnist-partitions/2-partition)에서 다운로드할 수 있습니다.

```
gsutil cp -r gs://tff-datasets-public/emnist-partitions/2-partition
```

그런 다음 이를 복제본에 복사하여 각 포드에 업로드할 수 있습니다. 예를 들면 다음과 같습니다.

```
kubectl cp emnist_part_1.sqlite tff-workers-deployment-1-6bb8d458d5-hjl9d:/root/worker/data/emnist_partition.sqlite

kubectl cp emnist_part_2.sqlite tff-workers-deployment-2-6cb76c6f5d-hqt88:/root/worker/data/emnist_partition.sqlite
```

## 시뮬레이션 실행

이제 클러스터에 대해 시뮬레이션을 실행할 준비가 되었습니다.

### TFF 환경 설정

In [ ]:
#@test {"skip": true}
!pip install --quiet --upgrade tensorflow-federated
!pip install --quiet --upgrade nest-asyncio

import nest_asyncio
nest_asyncio.apply()

### 훈련 절차 정의

다음은 페더레이션 학습을 위한 데이터세트 반복 방법, 모델 아키텍처 및 라운드-오버-라운드 프로세스를 정의합니다. 자세한 내용은 [여기](https://www.tensorflow.org/federated/tutorials/loading_remote_data#training_the_model)에서 확인하세요.

In [ ]:
import collections
from typing import Any, Optional, List
import tensorflow as tf
import tensorflow_federated as tff


class FederatedData(tff.program.FederatedDataSource,
                    tff.program.FederatedDataSourceIterator):
  """Interface for interacting with the federated training data."""

  def __init__(self, type_spec: tff.FederatedType):
    self._type_spec = type_spec
    self._capabilities = [tff.program.Capability.RANDOM_UNIFORM]

  @property
  def federated_type(self) -> tff.FederatedType:
    return self._type_spec

  @property
  def capabilities(self) -> List[tff.program.Capability]:
    return self._capabilities

  def iterator(self) -> tff.program.FederatedDataSourceIterator:
    return self

  def select(self, num_clients: Optional[int] = None) -> Any:
    data_uris = [f'uri://{i}' for i in range(num_clients)]
    return tff.framework.CreateDataDescriptor(
        arg_uris=data_uris, arg_type=self._type_spec)


input_spec = collections.OrderedDict([
    ('x', tf.TensorSpec(shape=(1, 784), dtype=tf.float32, name=None)),
    ('y', tf.TensorSpec(shape=(1, 1), dtype=tf.int32, name=None))
])
element_type = tff.types.StructWithPythonType(
    input_spec, container_type=collections.OrderedDict)
dataset_type = tff.types.SequenceType(element_type)

train_data_source = FederatedData(type_spec=dataset_type)
train_data_iterator = train_data_source.iterator()

def model_fn():
  model = tf.keras.models.Sequential([
      tf.keras.layers.InputLayer(input_shape=(784,)),
      tf.keras.layers.Dense(units=10, kernel_initializer='zeros'),
      tf.keras.layers.Softmax(),
  ])
  return tff.learning.from_keras_model(
      model,
      input_spec=input_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])


trainer = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))


def train_loop(num_rounds=10, num_clients=10):
  state = trainer.initialize()
  for round in range(1, num_rounds + 1):
    train_data = train_data_iterator.select(num_clients)
    result = trainer.next(state, train_data)
    state = result.state
    train_metrics = result.metrics['client_work']['train']
    print('round {:2d}, metrics={}'.format(round, train_metrics))

### TFF 작업자에 연결

기본적으로 TFF는 모든 계산을 로컬에서 실행합니다. 이 단계에서는 위에서 설정한 Kubernetes 서비스에 연결하도록 TFF에 지시합니다. 여기에 서비스의 외부 IP 주소를 복사해야 합니다.

In [ ]:
import grpc

ip_address_1 = '0.0.0.0'  #@param {type:"string"}
ip_address_2 = '0.0.0.0'  #@param {type:"string"}
port = 80

channels = [
    grpc.insecure_channel(f'{ip_address_1}:{port}'),
    grpc.insecure_channel(f'{ip_address_2}:{port}')
]

tff.backends.native.set_remote_python_execution_context(channels)

### 훈련 실행

In [ ]:
train_loop()

round  1, metrics=OrderedDict([('sparse_categorical_accuracy', 0.10557769), ('loss', 12.475689), ('num_examples', 5020), ('num_batches', 5020)])
round  2, metrics=OrderedDict([('sparse_categorical_accuracy', 0.11940298), ('loss', 10.497084), ('num_examples', 5360), ('num_batches', 5360)])
round  3, metrics=OrderedDict([('sparse_categorical_accuracy', 0.16223507), ('loss', 7.569645), ('num_examples', 5190), ('num_batches', 5190)])
round  4, metrics=OrderedDict([('sparse_categorical_accuracy', 0.2648384), ('loss', 6.0947175), ('num_examples', 5105), ('num_batches', 5105)])
round  5, metrics=OrderedDict([('sparse_categorical_accuracy', 0.29003084), ('loss', 6.2815433), ('num_examples', 4865), ('num_batches', 4865)])
round  6, metrics=OrderedDict([('sparse_categorical_accuracy', 0.40237388), ('loss', 4.630901), ('num_examples', 5055), ('num_batches', 5055)])
round  7, metrics=OrderedDict([('sparse_categorical_accuracy', 0.4288425), ('loss', 4.2358975), ('num_examples', 5270), ('num_batches